# Import libraries

In [6]:
import numpy as np
import pandas as pd
import skforecast
from skforecast.ForecasterAutoreg import ForecasterAutoreg
from statsmodels.tsa.stattools import adfuller


import warnings
import logging

# Suppress all warnings
warnings.filterwarnings("ignore")

# Suppress all logging
logging.disable(logging.CRITICAL)

# Dataset

Merge all the informations to build a serie id

In [7]:
df_train = pd.read_csv('train.csv')

# serie_name will be the id of the serie
df_train["serie_name"] = pd.concat([df_train["distribution_channel"], df_train["customer"], df_train["brand"], df_train["channel"], df_train["store_format"], df_train["category"]], axis=1).apply(lambda x: f"{x[0]}_{x[1]}_{x[2]}_{x[3]}_{x[4]}_{x[5]}", axis=1)

# Convert to datetime
df_train['date'] = pd.to_datetime(df_train[['year', 'month']].assign(day=1))

# Build multiindex
df_train.set_index(["serie_name",'date'], inplace=True)

#df_train.drop(["distribution_channel", "customer", "brand", "channel", "store_format", "category"], axis=1, inplace=True)

In [18]:
idxs = df_train.index.get_level_values(0).unique()

In [7]:
#df_train.drop(["year", "month"], axis=1, inplace=True)

In [9]:
df_train

volume_kg  distribution_channel  \
serie_name             date                                             
0_1_1_FOOD SERVICE_3_3 2010-01-01    460.026585                     0   
                       2010-02-01    456.944328                     0   
                       2010-03-01    926.790243                     0   
                       2010-04-01    762.549167                     0   
                       2010-05-01   1132.060704                     0   
...                                         ...                   ...   
1_15_1_DISCOUNTS_1_10  2019-12-01    938.077344                     1   
                       2020-01-01    464.815891                     1   
                       2020-02-01    471.421097                     1   
                       2020-03-01  12611.841468                     1   
                       2020-04-01    277.491080                     1   

                                   customer  brand       channel  \
serie_name             date                                        
0_1_1_FOOD SERVICE_3_3 2010-01-01         1      1  FOOD SERVICE   
                       2010-02-01         1      1  FOOD SERVICE   
                       2010-03-01         1      1  FOOD SERVICE   
                       2010-04-01         1      1  FOOD SERVICE   
                       2010-05-01         1      1  FOOD SERVICE   
...                                     ...    ...           ...   
1_15_1_DISCOUNTS_1_10  2019-12-01        15      1     DISCOUNTS   
                       2020-01-01        15      1     DISCOUNTS   
                       2020-02-01        15      1     DISCOUNTS   
                       2020-03-01        15      1     DISCOUNTS   
                       2020-04-01        15      1     DISCOUNTS   

                                   store_format  category  year  month  
serie_name             date                                             
0_1_1_FOOD SERVICE_3_3 2010-01-01             3         3  2010      1  
                       2010-02-01             3         3  2010      2  
                       2010-03-01             3         3  2010      3  
                       2010-04-01             3         3  2010      4  
                       2010-05-01             3         3  2010      5  
...                                         ...       ...   ...    ...  
1_15_1_DISCOUNTS_1_10  2019-12-01             1        10  2019     12  
                       2020-01-01             1        10  2020      1  
                       2020-02-01             1        10  2020      2  
                       2020-03-01             1        10  2020      3  
                       2020-04-01             1        10  2020      4  

[5793 rows x 9 columns]

Build External Features:

## Best Baseline Code:

In [11]:
import pandas as pd
from prophet import Prophet
from tqdm import tqdm
import numpy as np

def prepare_data_prophet(ts):
    ts = ts.asfreq('MS')
    ts.interpolate(inplace=True)
    
    # Apply log transformation
    ts['volume_kg'] = np.log(ts['volume_kg'])
    
    data_tr = ts

    # Add rolling statistics
    data_tr['rolling_mean_3'] = data_tr['volume_kg'].rolling(window=3).mean()
    data_tr['rolling_std_3'] = data_tr['volume_kg'].rolling(window=3).std()

    data_tr['rolling_mean_10'] = data_tr['volume_kg'].rolling(window=10).mean()
    data_tr['rolling_std_10'] = data_tr['volume_kg'].rolling(window=10).std()

    data_tr['rolling_mean_15'] = data_tr['volume_kg'].rolling(window=15).mean()
    data_tr['rolling_std_15'] = data_tr['volume_kg'].rolling(window=15).std()

    # Add difference features
    data_tr['volume_diff1'] = data_tr['volume_kg'].diff(1)

    data_tr.reset_index(inplace=True)
    data_tr.rename(columns={'date': 'ds', 'volume_kg': 'y'}, inplace=True)

    data_tr.dropna(inplace=True)
    
    return data_tr

def train_and_predict_prophet(ts, periods=4):
    ts_prepared = prepare_data_prophet(ts)
    
    model = Prophet(
        yearly_seasonality=True,
        daily_seasonality=False,
        changepoint_prior_scale=0.05,
        seasonality_prior_scale=10.0
    )
    
    # Adding additional seasonalities
    model.add_seasonality(name='monthly', period=30.5, fourier_order=5)
    model.add_seasonality(name='weekly', period=7, fourier_order=3)

    # Adding month and year as regressors
    model.add_regressor('month')
    model.add_regressor('year')

    model.fit(ts_prepared)
    
    future = model.make_future_dataframe(periods=periods, freq='MS')
    forecast = model.predict(future)

    # Inverse the log transformation
    forecast['yhat'] = np.exp(forecast['yhat'])

    return forecast[['ds', 'yhat']].tail(periods)

def create_submission_prophet(df_train, periods=4):
    unique_series = df_train.index.get_level_values(0).unique()
    all_predictions = []

    progress_bar = tqdm(total=len(unique_series))
    progress_bar.set_description("Evaluating series")

    for series in unique_series:
        ts = df_train.loc[series]
        predictions = train_and_predict_prophet(ts, periods)
        
        for _, row in predictions.iterrows():
            all_predictions.append({
                'id': f"{row['ds'].year}_{row['ds'].month}_{series}",
                'volume_kg': row['yhat']
            })
        progress_bar.update(1)
    
    submission_df = pd.DataFrame(all_predictions)
    submission_df.to_csv('submissionProphet12.csv', index=False)
    print("Submission file created with name 'submission.csv'")

## Adding Extra Regressors

In [12]:
import pandas as pd
from prophet import Prophet
from tqdm import tqdm
import numpy as np

def prepare_data_prophet(ts):
    ts = ts.asfreq('MS')
    ts.interpolate(inplace=True)
    
    # Apply log transformation
    ts['volume_kg'] = np.log(ts['volume_kg'])
    
    data_tr = ts

    # Add rolling statistics
    data_tr['rolling_mean_3'] = data_tr['volume_kg'].rolling(window=3).mean()
    data_tr['rolling_std_3'] = data_tr['volume_kg'].rolling(window=3).std()

    data_tr['rolling_mean_10'] = data_tr['volume_kg'].rolling(window=10).mean()
    data_tr['rolling_std_10'] = data_tr['volume_kg'].rolling(window=10).std()

    data_tr['rolling_mean_15'] = data_tr['volume_kg'].rolling(window=15).mean()
    data_tr['rolling_std_15'] = data_tr['volume_kg'].rolling(window=15).std()

    # Add difference features
    data_tr['volume_diff1'] = data_tr['volume_kg'].diff(1)

    data_tr.reset_index(inplace=True)
    data_tr.rename(columns={'date': 'ds', 'volume_kg': 'y'}, inplace=True)

    # Add month and year as regressors
    data_tr['month'] = data_tr['ds'].dt.month
    data_tr['year'] = data_tr['ds'].dt.year

    data_tr.dropna(inplace=True)
    
    return data_tr

def train_and_predict_prophet(ts, periods=4):
    ts_prepared = prepare_data_prophet(ts)
    
    model = Prophet(
        yearly_seasonality=True,
        daily_seasonality=False,
        changepoint_prior_scale=0.05,
        seasonality_prior_scale=10.0
    )
    
    # Adding additional seasonalities
    model.add_seasonality(name='monthly', period=30.5, fourier_order=5)
    model.add_seasonality(name='weekly', period=7, fourier_order=3)

    # Adding month and year as regressors
    model.add_regressor('month')
    model.add_regressor('year')

    model.fit(ts_prepared)
    
    future = model.make_future_dataframe(periods=periods, freq='MS')
    # Add future values for regressors
    future['month'] = future['ds'].dt.month
    future['year'] = future['ds'].dt.year

    forecast = model.predict(future)

    # Inverse the log transformation
    forecast['yhat'] = np.exp(forecast['yhat'])

    return forecast[['ds', 'yhat']].tail(periods)

def create_submission_prophet(df_train, periods=4):
    unique_series = df_train.index.get_level_values(0).unique()
    all_predictions = []

    progress_bar = tqdm(total=len(unique_series))
    progress_bar.set_description("Evaluating series")

    for series in unique_series:
        ts = df_train.loc[series]
        predictions = train_and_predict_prophet(ts, periods)
        
        for _, row in predictions.iterrows():
            all_predictions.append({
                'id': f"{row['ds'].year}_{row['ds'].month}_{series}",
                'volume_kg': row['yhat']
            })
        progress_bar.update(1)
    
    submission_df = pd.DataFrame(all_predictions)
    submission_df.to_csv('submissionProphet12.csv', index=False)
    print("Submission file created with name 'submission.csv'")

In [13]:
create_submission_prophet(df_train)

Evaluating series: 100%|██████████| 48/48 [00:25<00:00,  1.87it/s]

Submission file created with name 'submission.csv'


## Cross Validate

In [14]:
import pandas as pd
from prophet import Prophet
from tqdm import tqdm
import numpy as np
from sklearn.model_selection import TimeSeriesSplit
from sklearn.metrics import mean_squared_error

def prepare_data_prophet(ts):
    ts = ts.asfreq('MS')
    ts.interpolate(inplace=True)
    
    # Apply log transformation
    ts['volume_kg'] = np.log(ts['volume_kg'])
    
    data_tr = ts

    # Add rolling statistics
    data_tr['rolling_mean_3'] = data_tr['volume_kg'].rolling(window=3).mean()
    data_tr['rolling_std_3'] = data_tr['volume_kg'].rolling(window=3).std()

    data_tr['rolling_mean_10'] = data_tr['volume_kg'].rolling(window=10).mean()
    data_tr['rolling_std_10'] = data_tr['volume_kg'].rolling(window=10).std()

    data_tr['rolling_mean_15'] = data_tr['volume_kg'].rolling(window=15).mean()
    data_tr['rolling_std_15'] = data_tr['volume_kg'].rolling(window=15).std()

    # Add difference features
    data_tr['volume_diff1'] = data_tr['volume_kg'].diff(1)

    data_tr.reset_index(inplace=True)
    data_tr.rename(columns={'date': 'ds', 'volume_kg': 'y'}, inplace=True)

    # Add month and year as regressors
    data_tr['month'] = data_tr['ds'].dt.month
    data_tr['year'] = data_tr['ds'].dt.year

    data_tr.dropna(inplace=True)
    
    return data_tr

def train_and_predict_prophet(ts, periods=4):
    ts_prepared = prepare_data_prophet(ts)
    
    model = Prophet(
        yearly_seasonality=True,
        daily_seasonality=False,
        changepoint_prior_scale=0.05,
        seasonality_prior_scale=10.0
    )
    
    # Adding additional seasonalities
    model.add_seasonality(name='monthly', period=30.5, fourier_order=5)
    model.add_seasonality(name='weekly', period=7, fourier_order=3)

    # Adding month and year as regressors
    model.add_regressor('month')
    model.add_regressor('year')

    model.fit(ts_prepared)
    
    future = model.make_future_dataframe(periods=periods, freq='MS')
    # Add future values for regressors
    future['month'] = future['ds'].dt.month
    future['year'] = future['ds'].dt.year

    forecast = model.predict(future)

    # Inverse the log transformation
    forecast['yhat'] = np.exp(forecast['yhat'])

    return forecast[['ds', 'yhat']].tail(periods)

def cross_validate_prophet(ts, n_splits=5):
    tscv = TimeSeriesSplit(n_splits=n_splits)
    ts_prepared = prepare_data_prophet(ts)
    
    errors = []
    
    for train_index, test_index in tscv.split(ts_prepared):
        train_data = ts_prepared.iloc[train_index]
        test_data = ts_prepared.iloc[test_index]
        
        model = Prophet(
            yearly_seasonality=True,
            daily_seasonality=False,
            changepoint_prior_scale=0.001, 
            seasonality_prior_scale=20.0
        )
        
        model.add_seasonality(name='monthly', period=30.5, fourier_order=3)
        model.add_seasonality(name='weekly', period=7, fourier_order=2)
        
        # Adding month and year as regressors
        model.add_regressor('month')
        model.add_regressor('year')
        
        model.fit(train_data)
        
        future = model.make_future_dataframe(periods=len(test_data), freq='MS')
        future['month'] = future['ds'].dt.month
        future['year'] = future['ds'].dt.year
        
        forecast = model.predict(future)
        forecast['yhat'] = np.exp(forecast['yhat'])
        
        error = mean_squared_error(test_data['y'], forecast['yhat'][:len(test_data)])
        errors.append(error)
    
    return np.mean(errors), np.std(errors)

def create_submission_prophet(df_train, periods=4):
    unique_series = df_train.index.get_level_values(0).unique()
    all_predictions = []

    progress_bar = tqdm(total=len(unique_series))
    progress_bar.set_description("Evaluating series")

    for series in unique_series:
        ts = df_train.loc[series]
        mean_error, std_error = cross_validate_prophet(ts, n_splits=5)
        print(f'Series: {series} - Mean CV Error: {mean_error}, Std CV Error: {std_error}')
        
        predictions = train_and_predict_prophet(ts, periods)
        
        for _, row in predictions.iterrows():
            all_predictions.append({
                'id': f"{row['ds'].year}_{row['ds'].month}_{series}",
                'volume_kg': row['yhat']
            })
        progress_bar.update(1)
    
    submission_df = pd.DataFrame(all_predictions)
    submission_df.to_csv('submissionProphet12CV.csv', index=False)
    print("Submission file created with name 'submissionProphet12.csv'")


In [15]:
create_submission_prophet(df_train)

Evaluating series:   0%|          | 0/48 [00:00<?, ?it/s]

Series: 0_1_1_FOOD SERVICE_3_3 - Mean CV Error: 748908.7010479047, Std CV Error: 45593.57614000051


Evaluating series:   2%|▏         | 1/48 [00:07<05:54,  7.54s/it]

Series: 0_1_1_FOOD SERVICE_3_6 - Mean CV Error: 33012442.991395913, Std CV Error: 1226862.3081144663


Evaluating series:   4%|▍         | 2/48 [00:15<05:48,  7.57s/it]

...

Series: 1_14_3_MODERN TRADE_2_10 - Mean CV Error: 45849209.58036162, Std CV Error: 1700823.5161645906


Evaluating series:  98%|█████████▊| 47/48 [05:11<00:07,  7.78s/it]

Series: 1_15_1_DISCOUNTS_1_10 - Mean CV Error: 42247433.916891314, Std CV Error: 5152192.437920338


Evaluating series: 100%|██████████| 48/48 [05:21<00:00,  6.70s/it]

Submission file created with name 'submissionProphet12.csv'
